<a href="https://www.kaggle.com/code/anki112279/choropleth-for-canada-provinces?scriptVersionId=102772873" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Choropleth maps are used to create heatmaps based on the geographical boundries. 

A geoJson file is required that has geometries for the region in interest. 

Opendatasoft provides geometries of Canadian provinces and are used in the heatmaps.

https://data.opendatasoft.com/explore/dataset/georef-canada-province%40public/export/?disjunctive.prov_name_en


The notebook is divided into tow sections:

1. Calculates people who are living in unaffordable housing by years. 

2. Create heatmap by pvoinces.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime 
import regex as re


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Part -1 Data preparation 

Used population, housing and income data to calculate population who are living under unaffordable homes for each reigon. 

In [2]:
# pulls out income distribution data by population
income_df_base = pd.read_csv('/kaggle/input/housing-affordability-in-canada/income-distribution-2012-2020.csv')
income_df_base = income_df_base.dropna()

In [3]:
# import population data

pop_df_base = pd.read_csv('/kaggle/input/housing-affordability-in-canada/population-by-region-1946-2022.csv')
pop_df_base.head(10)

,REF_DATE,GEO,DGUID,VECTOR,COORDINATE,Population estimate
0,Jan-46,Canada,2016A000011124,v1,1,12188000
1,Apr-46,Canada,2016A000011124,v1,1,12241000
2,Jul-46,Canada,2016A000011124,v1,1,12316000
3,Oct-46,Canada,2016A000011124,v1,1,12393000
4,Jan-47,Canada,2016A000011124,v1,1,12450000
5,Apr-47,Canada,2016A000011124,v1,1,12507000
6,Jul-47,Canada,2016A000011124,v1,1,12576000
7,Oct-47,Canada,2016A000011124,v1,1,12646000
8,Jan-48,Canada,2016A000011124,v1,1,12710000
9,Apr-48,Canada,2016A000011124,v1,1,12773000


In [4]:
#pulls out housing and rental price by population
price_df_base  = pd.read_csv('/kaggle/input/housing-affordability-in-canada/housing-supply-price-rental.csv')

price_df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944 entries, 0 to 943
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Unnamed: 0                         944 non-null    int64  
 1   year                               944 non-null    float64
 2   total_dwelling                     944 non-null    int64  
 3   single_detached                    944 non-null    int64  
 4   multiple                           944 non-null    int64  
 5   semi_detached                      944 non-null    int64  
 6   row                                944 non-null    int64  
 7   apartment                          944 non-null    int64  
 8   total_dwelling_market              944 non-null    int64  
 9   homeownership_freehold             943 non-null    float64
 10  rental                             944 non-null    int64  
 11  homeownership_condo                944 non-null    int64  

In [5]:
# convert the REF_DATE to datetime and change it to yearly frequency

def expand_year(month_year):
    last_two_digits_year = month_year[-2:]
    if int(last_two_digits_year) > 22:
        month_year =  month_year[:-3] + ' 19' + month_year[-2:]
    else:
        month_year = month_year[:-3] + ' 20' + month_year[-2:]
    return datetime.strptime(str(month_year), '%b %Y')
        
# change to the datetime 

pop_df = pop_df_base.copy()

pop_df['REF_DATE'] = pop_df['REF_DATE'].apply(lambda x : expand_year(x))
pop_df = pop_df[pop_df['GEO'] == 'Canada']

pop_df['year'] = pd.DatetimeIndex(pop_df['REF_DATE']).year

pop_year_df = pop_df.groupby('year').agg({'Population estimate':max}).reset_index().rename({'Population estimate' : 'population'}, axis = 'columns')


In [6]:
# data preparation before merging with pop_year_df

income_df = income_df_base.copy()

income_df['year'] = income_df['year'].transform(lambda x: datetime.strptime(str(x)[:4], '%Y'))
income_df['year'] = pd.DatetimeIndex(income_df['year']).year

# remove commas in the numerical figures
income_df ['population_with_income']  = income_df ['population_with_income'].transform(lambda x : re.sub(",","", str(x)))

income_df  = income_df .astype({'population_with_income': int})


#income_pop = income_df.merge(income_w_pop, left_on  ='year', right_on = 'year', how = 'left')

In [7]:
# % of population have reported income 
income_pop = income_df.merge(pop_year_df, left_on  ='year', right_on = 'year', how = 'left')

income_pop['population_with_income_per'] =  income_pop['population_with_income']/income_pop['population']
income_pop = income_pop.drop(['population'], axis = 'columns')


In [8]:
income_pop.head(5)

,year,avg_employment_income,median_employment_income,population_with_income,0,5000,10000,20000,30000,40000,50000,60000,80000,100000,population_with_income_per
0,2012,"46,300","35,000",20070000,12.5,8.3,13.3,10.8,9.9,9.5,7.8,10.9,7.4,9.7,0.576128
1,2013,"47,300","35,500",20238000,12.4,7.9,13.1,11.0,10.0,9.3,7.6,11.1,7.0,10.5,0.574749
2,2014,"47,600","36,000",20598000,12.5,7.9,13.0,10.5,10.5,9.4,7.5,11.1,6.8,10.8,0.579262
3,2015,"47,800","36,000",20549000,12.1,7.6,13.4,10.9,10.0,9.1,7.8,11.0,7.5,10.5,0.573628
4,2016,"47,400","35,600",20619000,12.2,7.6,13.0,11.3,10.3,9.2,7.7,11.0,7.1,10.5,0.568663


In [9]:
#Adding rental prices to the dataframe
price_df = price_df_base.copy()

price_df['year'] = price_df['year'].transform(lambda x: datetime.strptime(str(x)[:4], '%Y'))
price_df['year'] = pd.DatetimeIndex(price_df['year']).year

In [10]:
# Merging price with income dataframes to calculate income distribution by region

df_price_income = price_df[['year','bachelor', 'population', 'region']].merge(income_pop, left_on = 'year', right_on = 'year', how = 'right')


In [11]:
# removing null values
df_price_income = df_price_income[~df_price_income.region.isnull()]
df_price_income.head(5)

,year,bachelor,population,region,avg_employment_income,median_employment_income,population_with_income,0,5000,10000,20000,30000,40000,50000,60000,80000,100000,population_with_income_per
0,2012,525.0,1250.265,manitoba,"46,300","35,000",20070000,12.5,8.3,13.3,10.8,9.9,9.5,7.8,10.9,7.4,9.7,0.576128
1,2012,864.0,2411.326,vancouver,"46,300","35,000",20070000,12.5,8.3,13.3,10.8,9.9,9.5,7.8,10.9,7.4,9.7,0.576128
2,2012,527.0,759.403,winnipeg,"46,300","35,000",20070000,12.5,8.3,13.3,10.8,9.9,9.5,7.8,10.9,7.4,9.7,0.576128
3,2012,776.0,1304.711,calgary,"46,300","35,000",20070000,12.5,8.3,13.3,10.8,9.9,9.5,7.8,10.9,7.4,9.7,0.576128
4,2012,516.0,145.080,prince_edward,"46,300","35,000",20070000,12.5,8.3,13.3,10.8,9.9,9.5,7.8,10.9,7.4,9.7,0.576128


In [12]:
pop_in_income_bracket = df_price_income.copy()
# finding population for each income bracket

cols = ['0', '5000','10000', '20000', '30000', '40000', '50000', '60000', '80000', '100000']

for col in cols:
    pop_in_income_bracket[col]  = pop_in_income_bracket[col]*pop_in_income_bracket['population']*pop_in_income_bracket['population_with_income_per']
    pop_in_income_bracket       = pop_in_income_bracket.astype({col:int})

In [13]:
# % of population living on the unaffordable home

pop_in_income_bracket["annual_rent"] = pop_in_income_bracket["bachelor"] * 12

In [14]:
def max_income_category(rent):
    income_ranges = [0, 5000,10000, 20000, 30000, 40000, 50000, 60000, 80000, 100000]
    income_range_sum = 0
    i = 0
    while rent >  income_ranges[i]:
        i+=1
        income_range_sum += 1
    return income_range_sum

In [15]:
pop_in_income_bracket.head(5)

,year,bachelor,population,region,avg_employment_income,median_employment_income,population_with_income,0,5000,10000,20000,30000,40000,50000,60000,80000,100000,population_with_income_per,annual_rent
0,2012,525.0,1250.265,manitoba,"46,300","35,000",20070000,9003,5978,9580,7779,7131,6842,5618,7851,5330,6987,0.576128,6300.0
1,2012,864.0,2411.326,vancouver,"46,300","35,000",20070000,17365,11530,18476,15003,13753,13197,10836,15142,10280,13475,0.576128,10368.0
2,2012,527.0,759.403,winnipeg,"46,300","35,000",20070000,5468,3631,5818,4725,4331,4156,3412,4768,3237,4243,0.576128,6324.0
3,2012,776.0,1304.711,calgary,"46,300","35,000",20070000,9396,6238,9997,8118,7441,7140,5863,8193,5562,7291,0.576128,9312.0
4,2012,516.0,145.080,prince_edward,"46,300","35,000",20070000,1044,693,1111,902,827,794,651,911,618,810,0.576128,6192.0


In [16]:
pop_in_income_bracket[cols].iloc[:, :2].sum(axis = 1)

0       14981
1       28895
2        9099
3       15634
4        1737
        ...  
169      4175
170    157441
171     15679
172      4794
173      1928
Length: 174, dtype: int64

In [17]:
pop_in_income_bracket['pop_under_unaffordable'] =  pop_in_income_bracket[cols].iloc[:, :2].sum(axis = 1)

In [18]:
pop_in_income_bracket.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   year                        174 non-null    int64  
 1   bachelor                    174 non-null    float64
 2   population                  174 non-null    float64
 3   region                      174 non-null    object 
 4   avg_employment_income       174 non-null    object 
 5   median_employment_income    174 non-null    object 
 6   population_with_income      174 non-null    int64  
 7   0                           174 non-null    int64  
 8   5000                        174 non-null    int64  
 9   10000                       174 non-null    int64  
 10  20000                       174 non-null    int64  
 11  30000                       174 non-null    int64  
 12  40000                       174 non-null    int64  
 13  50000                       174 non

### Part -2 Draw choropleth graph for provinces by number of people living in unaffordable homes

In [19]:
!pip install pdpipe
import pdpipe as pdp
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 845.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for decore: filename=decore-0.0.1-py2.py3-none-any.whl size=4191 sha256=25c0fc40f74d1cb6f78bafeb820c3e34a8fb308396d44b1b0425df82391ac532
  Stored in directory: /root/.cache/pip/wheels/71/8f/e1/b37357faaa92c7fbd2a8f735f96fcf032df04bcd3875465a3f
Successfully built decore


### Downloaded geoJson file from the below website:

https://data.opendatasoft.com/explore/dataset/georef-canada-province%40public/export/?disjunctive.prov_name_en

In [70]:
import plotly.express as px
import pandas as pd
import geopandas as gpd

#prov_data = gpd.read_file("/kaggle/input/canadaprovincesgeojsonfile/Canada-provinces.json")


prov_data = gpd.read_file("/kaggle/input/canadageojson/georef-canada-province.geojson")

In [63]:
prov_data.head(10)

,prov_type,prov_code,prov_name_fr,prov_name_en,year,prov_area_code,geometry
0,province,48,Alberta,Alberta,2019,CAN,"POLYGON ((-120.00000 60.00000, -120.00000 58.7..."
1,province,24,Québec,Quebec,2019,CAN,"MULTIPOLYGON (((-64.52477 60.29981, -64.58198 ..."
2,province,12,Nouvelle-Écosse,Nova Scotia,2019,CAN,"MULTIPOLYGON (((-64.27499 45.84152, -64.24538 ..."
3,province,47,Saskatchewan,Saskatchewan,2019,CAN,"POLYGON ((-110.00000 60.00000, -110.00033 59.7..."
4,province,35,Ontario,Ontario,2019,CAN,"MULTIPOLYGON (((-95.15344 49.64535, -95.10444 ..."
5,territory / territoire,60,Yukon,Yukon,2019,CAN,"MULTIPOLYGON (((-136.46860 68.86875, -136.5875..."
6,territory / territoire,62,Nunavut,Nunavut,2019,CAN,"MULTIPOLYGON (((-74.00000 64.53680, -74.04001 ..."
7,territory / territoire,61,Territoires du Nord-Ouest,Northwest Territories,2019,CAN,"MULTIPOLYGON (((-102.00000 60.00000, -102.0000..."
8,province,46,Manitoba,Manitoba,2019,CAN,"POLYGON ((-88.98649 56.84705, -89.17213 56.875..."
9,province,11,Île-du-Prince-Édouard,Prince Edward Island,2019,CAN,"POLYGON ((-61.98301 46.45775, -61.99012 46.464..."


In [64]:
pop_in_income_bracket.head(10)

,year,bachelor,population,region,avg_employment_income,median_employment_income,population_with_income,0,5000,10000,20000,30000,40000,50000,60000,80000,100000,population_with_income_per,annual_rent,pop_under_unaffordable
0,2012,525.0,1250.265,manitoba,"46,300","35,000",20070000,9003,5978,9580,7779,7131,6842,5618,7851,5330,6987,0.576128,6300.0,14981
1,2012,864.0,2411.326,vancouver,"46,300","35,000",20070000,17365,11530,18476,15003,13753,13197,10836,15142,10280,13475,0.576128,10368.0,28895
2,2012,527.0,759.403,winnipeg,"46,300","35,000",20070000,5468,3631,5818,4725,4331,4156,3412,4768,3237,4243,0.576128,6324.0,9099
3,2012,776.0,1304.711,calgary,"46,300","35,000",20070000,9396,6238,9997,8118,7441,7140,5863,8193,5562,7291,0.576128,9312.0,15634
4,2012,516.0,145.080,prince_edward,"46,300","35,000",20070000,1044,693,1111,902,827,794,651,911,618,810,0.576128,6192.0,1737
5,2012,754.0,966.617,ottawa,"46,300","35,000",20070000,6961,4622,7406,6014,5513,5290,4343,6070,4121,5401,0.576128,9048.0,11583
6,2012,684.0,373.817,oshawa,"46,300","35,000",20070000,2692,1787,2864,2325,2132,2045,1679,2347,1593,2089,0.576128,8208.0,4479
7,2012,390.0,160.233,saguenay,"46,300","35,000",20070000,1153,766,1227,996,913,876,720,1006,683,895,0.576128,4680.0,1919
8,2012,569.0,750.684,hamilton,"46,300","35,000",20070000,5406,3589,5752,4670,4281,4108,3373,4714,3200,4195,0.576128,6828.0,8995
9,2012,559.0,176.645,abbotsford,"46,300","35,000",20070000,1272,844,1353,1099,1007,966,793,1109,753,987,0.576128,6708.0,2116


In [74]:
prov_data.prov_name_fr.unique()

array(['Alberta', 'Quebec', 'Nouvelle-Écosse', 'Saskatchewan', 'Ontario',
       'Yukon', 'Nunavut', 'Territoires du Nord-Ouest', 'Manitoba',
       'Île-du-Prince-Édouard', 'Nouveau-Brunswick',
       'Terre-Neuve-et-Labrador', 'Colombie-Britannique'], dtype=object)

In [76]:
prov_data.prov_name_fr = prov_data.prov_name_fr.replace(['Québec'], 'Quebec') \
                    .replace(['Île-du-Prince-Édouard'] , 'Prince Edward Island',)\
                    .replace(['Nouveau-Brunswick'] , 'New Brunswick', )\
                    .replace(['Terre-Neuve-et-Labrador'] , 'Newfoundland and Labrador', )\
                    .replace(['Colombie-Britannique'] , 'British Columbia', )

In [110]:
prov_data.head(2)

,prov_type,prov_code,prov_name_fr,prov_name_en,year,prov_area_code,geometry
0,province,48,Alberta,Alberta,2019,CAN,"POLYGON ((-120.00000 60.00000, -120.00000 58.7..."
1,province,24,Quebec,Quebec,2019,CAN,"MULTIPOLYGON (((-64.52477 60.29981, -64.58198 ..."


In [78]:
# selecing only the provinces
regions = ['manitoba',  'vancouver', 'prince_edward','new_brunswick','saskatchewan','nova_scotia', 'quebec', 'alberta', 'ontario', 'saint_john']

pop_region = pop_in_income_bracket[pop_in_income_bracket.region.isin(regions)]



#convert provinces/region name to match with prov_date region name
regions = {
    'vancouver' : 'British Columbia',
    'alberta' : 'Alberta',
    'manitoba' : 'Manitoba', 
    'saskatchewan' :  'Saskatchewan',
    'ontario' :  'Ontario',
    'quebec' : 'Quebec',
    'prince_edward' : 'Prince Edward Island',
    'new_brunswick' : 'New Brunswick',
    'saint_john' : 'Newfoundland and Labrador'
    
}


In [79]:
def set_size(value):
    result = np.log(1+value/100)
    if result < 0:
        result = 0.01
    return result

In [80]:
pipeline = pdp.PdPipeline([
    pdp.ApplyByCols('pop_under_unaffordable', set_size, 'size', drop=False), 
    pdp.MapColVals('region', regions)])

dfk = pipeline.apply(pop_region)
dfk.fillna(0, inplace=True)

In [81]:
# merge with the geocode data

data_w_geocode = prov_data.merge(dfk, left_on = ['prov_name_en'], right_on = ['region'], how = 'left', )

In [109]:
import folium

# initialize the map and store it in a m object
m = folium.Map(location=[50, -65], zoom_start=3)

data = data_w_geocode[data_w_geocode.year_y == 2016]
data = data.astype({'size' : int, 'id':str})
folium.Choropleth(
    geo_data= prov_data,
    data=data,
    name="choropleth",
    columns= ["prov_name_en","pop_under_unaffordable"],
    key_on = "feature.properties.prov_name_en",
    fill_color="YlGn",
    fill_opacity=0.5,
    line_opacity=.1,
    legend_name="Unaffordability over the region",
).add_to(m)

folium.LayerControl().add_to(m)

m